## Importing libraries

In [6]:
!pip install unsloth

  Using cached unsloth-2025.3.18-py3-none-any.whl.metadata (46 kB)
  Using cached unsloth_zoo-2025.3.16-py3-none-any.whl.metadata (8.0 kB)
  Using cached xformers-0.0.29.post3-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (1.0 kB)
  Using cached bitsandbytes-0.45.3-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
  Using cached transformers-4.50.0-py3-none-any.whl.metadata (39 kB)
  Using cached trl-0.15.2-py3-none-any.whl.metadata (11 kB)
  Using cached cut_cross_entropy-25.1.1-py3-none-any.whl.metadata (9.3 kB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
  Using cached torchvision-0.21.0-cp310-cp310-manylinux1_x86_64.whl.metadata (6.1 kB)
Using cached unsloth-2025.3.18-py3-none-any.whl (192 kB)
Using cached transformers-4.50.0-py3-none-any.whl (10.2 MB)
Using cached trl-0.15.2-py3-none-any.whl (318 kB)
Using cached unsloth_zoo-2025.3.16-py3-none-any.whl (126 kB)
Using cached x

In [ ]:
import os
os.environ["HF_TOKEN"] = input("Enter your Hugging Face Token: ")
##  # The Token Genration

In [4]:
# deepseek-ai/deepseek-math-7b-instruct
# deepseek-ai/DeepSeek-R1-Distill-Llama-8B

In [4]:
## Large langugae model uses the 32 bits to store the model by using qunatization they reduce them to 4 or 8 bit qunatization
from unsloth import FastLanguageModel ## A very good library around the hugging face that helps to load model by using often in reduced precision (e.g., 4-bit quantization).
max_seq_length = 2048 ## Maximum sequence for the tokenize input
dtype = None ## Data type of the model weights use the default precision
load_in_4bit = True ## Enable 4 bit qunatization

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="deepseek-ai/DeepSeek-R1-Distill-Llama-8B", ## Pretrained Model from the hugging face
    max_seq_length=max_seq_length,## Set Maximum as set
    dtype=dtype,## Use default as it is set to none
    load_in_4bit=load_in_4bit,## Use 4 bit quantization to load the model
    token=os.getenv("HF_TOKEN"),  # add your HF token
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [6]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a math expert with advanced knowledge in mathematical reasoning, problem-solving, and theoretical concepts.
Please answer the following math question. You have to Give the Answer and solve The Response is like how to give the Answer
like this 'Response': 'give the value of every Emojies used like 🍎 = 4' for example 

### Question:
{}

### Response:
<think>{}"""

In [7]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load dataset from CSV
dataset = load_dataset("csv", data_files="/kaggle/input/emojimath/emoji_math.csv")

# Initialize tokenizer (replace with your model name)

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

# Define your training prompt style
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a math expert with advanced knowledge in mathematical reasoning, problem-solving, and theoretical concepts.
Please answer the following math question. You have to Give the Answer and solve The Response is like how to give the Answer
like this 'Response': 'give the value of every Emojies used like 🍎 = 4' for example 

### Question:
{}

### Response:
<think>
{}
</think>
{}""" + EOS_TOKEN

# Formatting function
def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output)
        texts.append(text)
    return {"text": texts}

# Apply formatting function
dataset = dataset.map(formatting_prompts_func, batched=True)

# Check formatted data
print(dataset["train"][0])


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

{'Question': '🍎 + 🍎 + 🍎 = 12', 'Complex_CoT': 'Break it down: 🍎 + 🍎 + 🍎 = 12 → Divide both sides by 3 → 🍎 = 4', 'Response': '🍎 = 4', 'text': "Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request. \nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a math expert with advanced knowledge in mathematical reasoning, problem-solving, and theoretical concepts.\nPlease answer the following math question. You have to Give the Answer and solve The Response is like how to give the Answer\nlike this 'Response': 'give the value of every Emojies used like 🍎 = 4' for example \n\n### Question:\n🍎 + 🍎 + 🍎 = 12\n\n### Response:\n<think>\nBreak it down: 🍎 + 🍎 + 🍎 = 12 → Divide both sides by 3 → 🍎 = 4\n</think>\n🍎 = 4<｜end▁of▁sentence｜>"}


In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r=8,  # Lower rank for faster updates
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=8,  # Lower influence speeds up training
    lora_dropout=0.1,  # Small dropout to improve generalization
    bias="none",
    use_gradient_checkpointing=False,  # Turn off for faster training
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset['train'],
    dataset_text_field="text",
    max_seq_length=256,  # Reduce sequence length for faster training
    dataset_num_proc=1,  # Reduce CPU overhead
    args=TrainingArguments(
        per_device_train_batch_size=4,  # Increase batch size if GPU allows
        gradient_accumulation_steps=2,  # Reduce accumulation to speed up steps
        warmup_steps=1,  # Reduce warmup
        max_steps=3,  # Reduce total steps for quick testing
        learning_rate=5e-4,  # Slightly higher learning rate for faster convergence
        fp16=True,  # Force fp16 if supported
        bf16=False,  # Disable bf16 for stability
        logging_steps=1,  # Log more frequently for monitoring
        optim="adamw_torch",  # Faster optimizer
        weight_decay=0.01,
        lr_scheduler_type="cosine",  # Cosine scheduler adapts better
        seed=3407,
        output_dir="outputs",
        report_to="none"
    ),
)


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.18 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Unsloth: Tokenizing ["text"]:   0%|          | 0/30 [00:00<?, ? examples/s]

In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 30 | Num Epochs = 1 | Total steps = 3
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 20,971,520/8,000,000,000 (0.26% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.607500
2,3.654000
3,3.067700


In [10]:
question = "🍎 + 🍌 = -12"


FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1500,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])



<think>
Alright, so I've got this math problem here: 🍎 + 🍌 = -12. Hmm, okay, let me try to figure this out. First off, I know that in math problems, especially algebraic ones, each symbol usually stands for a specific number. In this case, the emojis are probably representing numbers. 

So, 🍎 is an apple emoji, and 🍌 is a pear emoji. In math terms, they might be variables like A and P, right? So, maybe A + P = -12. But I'm not sure if that's the case here. The problem doesn't specify what the emojis represent, so I have to make an assumption. 

Wait, maybe the emojis are just there to make it more fun, and their actual values are numbers. Like, 🍎 could be 9 and 🍌 could be 3, adding up to 12, but since it's negative, maybe they represent negative numbers. Or perhaps each emoji stands for a digit, like 1 and 2, but again, with a negative sign.

Let me think differently. Maybe it's a word problem where the emojis are part of words. For example, "apple" could be represented by A, and "pea

## Pushing to The Hugging face

In [14]:
!pip install huggingface_hub


In [16]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/usr/local/bin/huggingface-c

In [ ]:
from huggingface_hub import login
login(token="Your Hugging face token")  # Replace with your token

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [22]:
from huggingface_hub import HfApi
model_path = "DeepSeek-R1-Medical-COT-Merged"  # Your local model directory
repo_name = "Ahmed12121231312312312/DeepSeekr1fineTune"  # Change to your HF username/repo name
# Push the model
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)
print(f"Model uploaded to: https://huggingface.co/{repo_name}")

  0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Saved model to https://huggingface.co/Ahmed12121231312312312/DeepSeekr1fineTune


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Model uploaded to: https://huggingface.co/Ahmed12121231312312312/DeepSeekr1fineTune


In [4]:
pip install -U bitsandbytes

Note: you may need to restart the kernel to use updated packages.


In [6]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained("Ahmed12121231312312312/DeepSeekr1fineTune")
tokenizer = AutoTokenizer.from_pretrained("Ahmed12121231312312312/DeepSeekr1fineTune")

config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/132k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/231 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
from unsloth import FastLanguageModel
# Define model parameters
model_name = "Ahmed12121231312312312/DeepSeekr1fineTune"  # Replace with your Hugging Face repo
max_seq_length = 2048  # Adjust based on your model's training
dtype = None  # Use default precision
load_in_4bit = True  # Enable 4-bit quantization to reduce memory usage

# Load the fine-tuned model from Hugging Face
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token=os.getenv("HF_TOKEN"),  # Ensure you have your Hugging Face token set
    device_map="auto",  # Automatically assign layers between CPU and GPU
)
print("Model successfully loaded!")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

In [9]:
from unsloth import FastLanguageModel
max_seq_length = 2048 ## Maximum sequence for the tokenize input
dtype = None ## Data type of the model weights use the default precision
load_in_4bit = True ## Enable 4 bit qunatization
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Ahmed12121231312312312/DeepSeekr1fineTune",
    max_seq_length=2048,
    load_in_4bit=True,  # Efficient loading
)

==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1593: UserWarning: Current model requires 4194304.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

## Saving it to local Memory

In [19]:
new_model_local = "DeepSeek-R1-COT-Merged"
model.save_pretrained(new_model_local) 
tokenizer.save_pretrained(new_model_local)
model.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit",)

Unsloth: You have 2 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 19.66 out of 31.35 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 38%|███▊      | 12/32 [00:00<00:00, 26.61it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [00:21<00:00,  1.52it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving DeepSeek-R1-Medical-COT/pytorch_model-00001-of-00004.bin...
Unsloth: Saving DeepSeek-R1-Medical-COT/pytorch_model-00002-of-00004.bin...
Unsloth: Saving DeepSeek-R1-Medical-COT/pytorch_model-00003-of-00004.bin...
Unsloth: Saving DeepSeek-R1-Medical-COT/pytorch_model-00004-of-00004.bin...
Done.


### !!!!!!!!!!!!!!!!!!!!!!!!!!!! Deleting the files

In [26]:
# Clear output folder
import os

def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)

folder_path = '/kaggle/working/DeepSeek-R1-COT-Merged'
remove_folder_contents(folder_path)
os.rmdir(folder_path)


### Saving it to local Memory

In [11]:
merged_model_path = "DeepSeek-R1-COT-Merged"  # New folder for the merged model

# Merge and save as a standard 16-bit model
model.save_pretrained_merged(merged_model_path, tokenizer, save_method="merged_16bit")
tokenizer.save_pretrained(merged_model_path)


Unsloth: You have 2 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 19.61 out of 31.35 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 44%|████▍     | 14/32 [00:00<00:00, 26.41it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [00:19<00:00,  1.68it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving DeepSeek-R1-Medical-COT-Merged/pytorch_model-00001-of-00004.bin...
Unsloth: Saving DeepSeek-R1-Medical-COT-Merged/pytorch_model-00002-of-00004.bin...
Unsloth: Saving DeepSeek-R1-Medical-COT-Merged/pytorch_model-00003-of-00004.bin...
Unsloth: Saving DeepSeek-R1-Medical-COT-Merged/pytorch_model-00004-of-00004.bin...
Done.


('DeepSeek-R1-Medical-COT-Merged/tokenizer_config.json',
 'DeepSeek-R1-Medical-COT-Merged/special_tokens_map.json',
 'DeepSeek-R1-Medical-COT-Merged/tokenizer.json')

In [12]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

# Path to the fine-tuned model
model_path = "/kaggle/working/DeepSeek-R1-COT-Merged"  

# Load the fine-tuned model with 4-bit quantization
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=2048,
    load_in_4bit=True,  # Efficient loading
)


==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
FastLanguageModel.for_inference(model)

# Define the prompt template
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a math expert with advanced knowledge in mathematical reasoning, problem-solving, and theoretical concepts.
Please answer the following math question. You have to Give the Answer and solve The Response is like how to give the Answer
like this 'Response': 'give the value of every Emojies used like 🍎 = 4' for example  

### Question:
{}

### Response:
<think>{}"""

# Define a math question
question = "🍎 + 🍎 + 🍎 = 12"

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate response
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1500,  # Adjust token limit
    use_cache=True,
)

# Decode and display the response
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])  # Extract and print the model's answer


<think>
Alright, so I've got this math problem here: 🍎 + 🍎 + 🍎 = 12. Hmm, okay, let me try to figure this out step by step. First, I need to understand what the problem is asking. It's adding three of the same emojis together to get 12. But what does each emoji represent numerically?

I know that emojis can sometimes be used to represent numbers or letters, especially in puzzles. For example, sometimes each emoji corresponds to a number, like 🍎 might be 1, 🍊 could be 2, and so on. So, if 🍎 is 1, then adding three of them would be 1 + 1 + 1, which is 3, but the result here is 12. That doesn't match, so maybe each emoji represents a different number.

Wait, maybe I should think about the individual values of the emojis. Let me list out what each emoji typically stands for. 🍎 is an apple, 🍎. 🍎 is a fruit, so maybe it's 1, but as I saw before, that doesn't add up to 12. Maybe it's something else.

Alternatively, perhaps the emojis are representing letters, and each letter corresponds to a